# Tooploox Data Science exercise

In [2]:
import pandas as pd

dataset = pd.read_csv('./data.csv', header=None)

### Analysing basic statistics for v(24), v(72), v(168)

In [3]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

for i in [24,72,168]:
    print "For %d-th hour: mean = %f, std = %f, median = %d" % (i, dataset[i].mean(), dataset[i].std(), dataset[i].median())

For 24-th hour: mean = 376765.517467, std = 923646.739985, median = 194357
For 72-th hour: mean = 613303.341703, std = 1653969.423316, median = 237417
For 168-th hour: mean = 743209.837336, std = 2007963.831213, median = 252287
